In [98]:
import tensorflow as tf
import tensorlayer as tl
import numpy as np
from tqdm import tqdm
from PIL import Image
import os

Loading data

In [101]:
def image_generator(folder, batch_size=1):
    path = os.path.join('../data/', folder)
    all_files = os.listdir(path)
    file_indices = np.random.choice(len(all_files), batch_size)
    to_return = []
    for i in tqdm(file_indices):
        rgb_img = Image.open(os.path.join('../data', folder, all_files[i]))
        ycbcr_img = rgb_img.convert('YCbCr')
        ycbcr_img = np.array(rgb_img.resize((224, 224), Image.LANCZOS))
        to_return.append(ycbcr_img)
    
    return to_return

def labels_generator(size, value=1):
    return np.array([value]*size)

def noise_generator(batch_size):
    noises = []
    for i in tqdm(range(batch_size)):
        noises.append(np.random.normal(size=(224, 224, 3)))
    
    return noises

Creating model and loading pre-trained weights

In [26]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 224, 224, 3])
x = tf.placeholder(tf.float32, [None, 1])
vgg16 = tl.models.VGG16(x, end_with='fc3_relu', reuse=None)
fc1 = tl.layers.DenseLayer(vgg16, n_units=256, act=tf.nn.leaky_relu, name='fc1')
fc2 = tl.layers.DenseLayer(fc1, n_units=1, act=tf.nn.sigmoid, name='fc2')

[TL] InputLayer  vgg16/input: (?, 224, 224, 3)
[TL] Conv2d vgg16/conv1_1: n_filter:64 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv1_2: n_filter:64 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv2_1: n_filter:128 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv2_2: n_filter:128 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv3_1: n_filter:256 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv3_2: n_filter:256 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv3_3: n_filter:256 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv4_1: n_filter:512 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv4_2: n_filter:512 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv4_3: n_filter:512 filter_size:(3, 3) strides:(1, 1) pad:SAME act:relu
[TL] Conv2d vgg16/conv5_1: n_filter

In [28]:
npz = np.load('vgg16_weights.npz')
params = []
for val in sorted(npz.items()):
    print("Loading params {}".format(str(val[1].shape)))
    params.append(val[1])

ops = []
for idx, param in enumerate(params):
    ops.append(vgg16.all_params[idx].assign(param))

del params

Loading params (3, 3, 3, 64)
Loading params (64,)
Loading params (3, 3, 64, 64)
Loading params (64,)
Loading params (3, 3, 64, 128)
Loading params (128,)
Loading params (3, 3, 128, 128)
Loading params (128,)
Loading params (3, 3, 128, 256)
Loading params (256,)
Loading params (3, 3, 256, 256)
Loading params (256,)
Loading params (3, 3, 256, 256)
Loading params (256,)
Loading params (3, 3, 256, 512)
Loading params (512,)
Loading params (3, 3, 512, 512)
Loading params (512,)
Loading params (3, 3, 512, 512)
Loading params (512,)
Loading params (3, 3, 512, 512)
Loading params (512,)
Loading params (3, 3, 512, 512)
Loading params (512,)
Loading params (3, 3, 512, 512)
Loading params (512,)
Loading params (25088, 4096)
Loading params (4096,)
Loading params (4096, 4096)
Loading params (4096,)
Loading params (4096, 1000)
Loading params (1000,)


In [102]:
# x_true = image_generator('train', 2048)
x_false = noise_generator(2048)
# y_true = labels_generator(2048)
# y_false = labels_generator(2048, value=0)


 86%|████████▌ | 1764/2048 [00:15<00:02, 116.19it/s]


100%|██████████| 2048/2048 [00:17<00:00, 116.34it/s]

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [103]:
a = np.concatenate((x_true, x_false), axis=0)
a.shape

ValueError: could not broadcast input array from shape (224,224,3) into shape (224,224)

In [ ]:
with tf.Session as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(100):
        x = np.random.shuffle(np.concatenate(x_true, x_false))
    

In [ ]:
tfgan = tf.contrib.gan